##relation wash
To make the interactive relation graph, we need to wash the data for it. We built the relation matrix, where the each column and row index means an artist. For example, if there's relation between artist no.2 and no.3. then matrix[2][5] and artist[5][2] with be set to a value larger than 0.

###relation wash prepare

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
import math

In [ ]:
artistBirth = pd.read_csv("artistBirth19th2.csv")
apprentice = pd.read_csv("Apprenticeship.csv")
movement = pd.read_csv("ArtistMovements.csv")
artwork = pd.read_csv("artwork.csv")
recommandation = pd.read_csv("Recommendation.csv")

In [ ]:
print(artistBirth.loc[0])
print(movement.loc[0])


id                            0
name           Vincent Van Gogh
birthplace                    0
birthdate            1853-03-30
birthplace1             Zundert
birthplace2         Netherlands
Name: 0, dtype: object
artist_id       0
movement_id    43
Name: 0, dtype: int64
43


In [ ]:
n=len(artistBirth)

In [ ]:
appren=pd.DataFrame(np.zeros((n, n)))
move=pd.DataFrame(np.zeros((n, n)))
recomm=pd.DataFrame(np.zeros((n, n)))
appren['id']=artistBirth['id']
move['id']=artistBirth['id']
recomm['id']=artistBirth['id']

In [ ]:
artists=artistBirth['id'].tolist()

###wash movement relation
Here the relation Matrix will be set to 1, if two Artists are belong to same Movement.

In [ ]:
movement['artist_id']=movement['artist_id'].astype(int)
movement['movement_id']=movement['movement_id'].astype(int)

In [ ]:
movement=movement[movement['artist_id'].isin(artists)]

In [ ]:
movement=movement.sort_values(by=['movement_id'],ignore_index=True)

In [ ]:
curMove=movement.loc[0][1]
preArtist = movement.loc[0][0]
preIndex=np.where(artistBirth["id"] == preArtist)[0]

In [ ]:
print(len(movement))
print(preArtist)

165
61


In [ ]:
for i in range(len(movement)):
  if movement.loc[i][1] == curMove:
    curArtist=movement.loc[i][0]
    curIndex=np.where(artistBirth["id"] == curArtist)[0]
    move.at[preIndex, curIndex] = 1
    move.at[curIndex, preIndex] = 1
    #move.loc[preIndex][curIndex]=1
    #move.loc[curIndex][preIndex]=1
  else:
    curMove=movement.loc[i][1]
    preArtist = movement.loc[i][0]
    preIndex=np.where(artistBirth["id"] == preArtist)[0]

In [ ]:
 move.to_csv('movementRelation.csv')

###wash apprentice relation
Here the relation Matrix will be set to 1, if two Artists have apprentice relation.

In [ ]:
apprentice = pd.read_csv("Apprenticeship.csv")
artistBirth = pd.read_csv("artistBirth19th2.csv")

In [ ]:
n=len(artistBirth)
appren=pd.DataFrame(np.zeros((n, n)))
appren['id']=artistBirth['id']

In [ ]:
apprentice['student_id']=apprentice['student_id'].astype(int)
apprentice['teacher_id']=apprentice['teacher_id'].astype(int)

In [ ]:
for i in range(len(apprentice)):
  curStudentId = apprentice.loc[i][0]
  curTeacherId = apprentice.loc[i][1]
  curStudentIndex = np.where(artistBirth["id"] == curStudentId)[0]
  curTeacherIndex = np.where(artistBirth["id"] == curTeacherId)[0]
  #print(curStudentIndex,curTeacherIndex )
  appren.at[curStudentIndex,curTeacherIndex]=1
  appren.at[curTeacherIndex,curStudentIndex]=1

In [ ]:
appren.to_csv('apprenticeRelation.csv')

### wash recommendation relation
The recommendation relation is actually between the Artworks. Thus between two artists, there might be multiple artworks that have relations. Thus we regard each pair of arts creat by diffent artists as 1, cumulate them to each artist. 

In [ ]:
print(recommandation.loc[0])
print(len(recommandation))
print(artwork.loc[0])

artwork            0
recommended    10013
Name: 0, dtype: int64
5774
id                                                           0
name                        Cafe Terrace on the Place du Forum
image_url    https://kuleuven-datathon-2023.s3.eu-central-1...
artist                                                       0
rating                                                     8.0
Name: 0, dtype: object


In [ ]:
artwork=artwork[artwork['artist'].isin(artists)]

In [ ]:
print(len(artwork))

2513


In [ ]:
artworks=artwork['id'].tolist()

In [ ]:
recommandation=recommandation[recommandation['artwork'].isin(artworks)]
recommandation=recommandation[recommandation['recommended'].isin(artworks)]

In [ ]:
print(recommandation)

      artwork  recommended
3           2         4174
6          12          878
7          15          732
8          15          900
9          19          472
...       ...          ...
5645    10260          369
5647    10263         4868
5651    10265         5207
5652    10266          117
5673    10337         4167

[497 rows x 2 columns]


In [ ]:
recommandation=recommandation.reset_index()

In [ ]:
print(recommandation)

     index  artwork  recommended
0        3        2         4174
1        6       12          878
2        7       15          732
3        8       15          900
4        9       19          472
..     ...      ...          ...
492   5645    10260          369
493   5647    10263         4868
494   5651    10265         5207
495   5652    10266          117
496   5673    10337         4167

[497 rows x 3 columns]


In [ ]:
print(artwork)

          id                                               name  \
0          0                 Cafe Terrace on the Place du Forum   
1          1                                       Starry Night   
2          2                                           A Digger   
3          3                                A Group of Cottages   
4          4                                    A Pair of Shoes   
...      ...                                                ...   
10320  10339        Portrait of Charles X (1757-1836) c.1824-30   
10321  10340  The Artist in His Room at the Villa Medici, Ro...   
10322  10341  Portrait of Achille-Etna Michallon (1796-1822)...   
10323  10342    The Wife of an Bandit Examining his Spoils 1825   
10324  10343                                 The Brigand's Wife   

                                               image_url  artist  rating  
0      https://kuleuven-datathon-2023.s3.eu-central-1...       0     8.0  
1      https://kuleuven-datathon-2023.s3.eu-c

In [ ]:
test=artwork.loc[artwork["id"] == 4]['artist'].iloc[0]

In [ ]:
test.iloc[0]

0

In [ ]:
print(type(test))

<class 'numpy.int64'>


In [ ]:
for i in range(len(recommandation)):
  id1 = recommandation.loc[i][1]
  id2 = recommandation.loc[i][2]
  idArtist1 = artwork.loc[artwork["id"] == id1]['artist'].iloc[0]
  idArtist2 = artwork.loc[artwork["id"] == id2]['artist'].iloc[0]
  indexArtist1=np.where(artistBirth["id"] == idArtist1)[0]
  indexArtist2=np.where(artistBirth["id"] == idArtist2)[0]
  if indexArtist1!= indexArtist2:
    recomm.at[indexArtist1,indexArtist2]=recomm.loc[indexArtist1][indexArtist2]+1
    recomm.at[indexArtist2,indexArtist1]=recomm.loc[indexArtist2][indexArtist1]+1
    print(indexArtist1,indexArtist2)

In [ ]:
recomm.to_csv('recommandationeRelation.csv')

In [ ]:

!pip install pyecharts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 KB 16.8 MB/s eta 0:00:00


##to the picture
With the help of relation Matrix, we seperatly visulization three relation graph. There is a drawback that the dots in the graph(which represent one artists) have a same size. Even the matrix of recommendation contain different numbers, the visulization still regard them as 1. Anyway to fix, we use Neo4j to make a graph with weights bsing on the frequency.

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
import math

In [ ]:
apprentice = pd.read_csv('apprenticeRelation.csv')
movement = pd.read_csv('movementRelation.csv')
recommendation = pd.read_csv('recommandationRelation.csv')

In [ ]:
artist  =pd.read_csv('artistBirth19th2.csv')

In [ ]:
import heapq
from pyecharts import options as opts
#from pyecharts import Style
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Graph, Timeline
import csv

In [ ]:
print(apprentice.iloc[16][4])

1


In [ ]:
allNodes=[]
allLinks=[]

In [ ]:
nodes1=[]
for i in range(len(apprentice)):
  if(apprentice.iloc[i]['value']>0 ):
    node={"id":artist.iloc[i]['name'],"name":artist.iloc[i]['name'],"size":apprentice.iloc[i]['value']}
    nodes1.append(node)
allNodes.append(nodes1)

In [ ]:
nodes2=[]
for i in range(len(movement)):
  if(movement.iloc[i]['value']>0 ):
    node={"id":artist.iloc[i]['name'],"name":artist.iloc[i]['name'],"size":movement.iloc[i]['value']}
    nodes2.append(node)
allNodes.append(nodes2)

In [ ]:
nodes3=[]
for i in range(len(recommendation)):
  if(recommendation.iloc[i]['value']>0 ):
    node={"id":artist.iloc[i]['name'],"name":artist.iloc[i]['name'],"size":recommendation.iloc[i]['value']}
    nodes3.append(node)
allNodes.append(nodes3)

In [ ]:
links1=[]
for i in range(len(apprentice)):
  for j in range(1,i+1):
    if(apprentice.iloc[i][j]>0):
      link={"source":artist.iloc[i]['name'],"target":artist.iloc[j-1]['name']}
      links1.append(link)
allLinks.append(links1)


In [ ]:
links2=[]
for i in range(len(movement)):
  for j in range(1,i+1):
    if(movement.iloc[i][j]>0):
      link={"source":artist.iloc[i]['name'],"target":artist.iloc[j-1]['name']}
      links2.append(link)
allLinks.append(links2)

In [ ]:
links3=[]
for i in range(len(recommendation)):
  for j in range(1,i+1):
    if(recommendation.iloc[i][j]>0):
      link={"source":artist.iloc[i]['name'],"target":artist.iloc[j-1]['name']}
      links3.append(link)
allLinks.append(links3)

In [ ]:
timeline = Timeline(init_opts=opts.InitOpts(width='1100px', height='600px')) 
timeline.add_schema(axis_type='category', orient='horizontal', symbol='circle', symbol_size=10, 
                    play_interval=8000, is_auto_play=False, is_loop_play=False, is_timeline_show=True, 
                    control_position='left', pos_left='10%', pos_bottom='5%', width='800px', height='40px',
                    linestyle_opts = opts.LineStyleOpts(width = 5,
                                   type_ = 'dashed',
                                   color = '#564537'),
                    label_opts = opts.LabelOpts(color = '#564537',
                                    font_size = 15),
                    itemstyle_opts=opts.ItemStyleOpts(color="#564537")

                    ) 

In [ ]:
graphList=["Apprentice","if in Same movement", "Artworks Recommendations"]

In [ ]:
for i in range(len(graphList)):
  graph = Graph(init_opts=opts.InitOpts( bg_color="#dfc5ae",width="1000px", height="500px"))
  graph.set_global_opts(title_opts=opts.TitleOpts(title="19th century Artists:Relations",
                                                subtitle="of {}".format(graphList[i]),
                                                title_textstyle_opts=opts.TextStyleOpts(color="#022955",font_weight="bold",font_size=30),
                                                subtitle_textstyle_opts=opts.TextStyleOpts(color="#022955",font_size=23)
                                                ))
  graph.add(series_name="",
          nodes=allNodes[i],
          links=allLinks[i],
          layout="circular",
          is_rotate_label=False,
          #color="#392e25",
          label_opts=opts.LabelOpts(is_show=True,color="#8B4513",position="top"),
          linestyle_opts=opts.LineStyleOpts(curve=0.3),
          repulsion=80)
  timeline.add(graph, "{}".format(graphList[i]))

In [ ]:
timeline.render("relation.html")

'/content/relation.html'